## Imports

In [ ]:
import itertools
import os
from os.path import join
import numpy as np
import pandas as pd
 
from scipy.signal import sosfiltfilt, butter
from scipy.spatial.transform import Rotation

from plotly.subplots import make_subplots
import plotly.graph_objects as go
import matplotlib.pyplot as plt

import pickle

## Data choice

In [ ]:
experiment = 'february'
fruit = 'orange'
fruit_labels = {'apple': 3, 'orange': 5, 'banana': 7}
fruit_label = fruit_labels[fruit]
cut_qualities = ['cut']

filter_data = True

desired_freq = 500

## Data extraction

In [ ]:
data_folder = join('../..', 'data', 'raw_data', experiment, fruit)
all_runs = {cq: [run for run in os.listdir(data_folder) if cq in run and run[-3:] == "csv"] for cq in cut_qualities}
print(all_runs)

In [ ]:
def filter_data(data, sensor_freq, cutoff_freq=10, order=2):
    '''Apply digital Butterworth filter with a cutoff frequency 'cutoff_freq' and order 'order'
    forward and backward to columns in 'data'.'''
    sos = butter(order, cutoff_freq, fs=sensor_freq, output='sos')
    return data.apply(lambda x: sosfiltfilt(sos, x), axis=0)
    
def data_average(df):
    '''Compute and return colum-average of data frame.'''
    return df.mean()
    
def downsample(data, time_vector, average_function, selected_columns):
    '''Downsample selected columns 'selected_colums' from data frame 'data'
    to sample points given by 'time_vecor' with a desired downsampling function 'average_function'
    and return the downsampled data frame.'''
    def insert_row(data, row, labels=None):
        return data.append(pd.Series(row, labels), ignore_index=True)
    
    current_time_index = 0
    downsampled_data = pd.DataFrame(columns=selected_columns)
    
    for i in range(len(time_vector)):
        t = time_vector.iloc[i]
        start_time = current_time_index
    
        while current_time_index < data.shape[0] and data['timestamp'].iloc[current_time_index] < t:
            current_time_index = current_time_index + 1
        stop_time = current_time_index
    
        average_data = average_function(data.iloc[start_time:stop_time][selected_columns]) if stop_time != start_time else np.empty(len(selected_columns)) * np.nan
        downsampled_data = insert_row(downsampled_data, average_data, selected_columns)
    
    downsampled_data = downsampled_data.dropna()
    downsampled_data = downsampled_data.reset_index()
    
    return downsampled_data

def export_run(run, data, folder_name):
    '''Export data from data frame 'data' to csv file.'''
    export_folder = join('../..', 'data', folder_name)
    
    if not os.path.isdir(export_folder):
        os.makedirs(export_folder)
        
    name = '_'.join(['segmented'] + run.split('_')[1:])
    data.to_csv(join(export_folder, name), index=False)

In [ ]:
ft_headers = list(itertools.chain.from_iterable((v + '_x', v + '_y', v + '_z') for v in ['ee_force']))
pos_headers = list(itertools.chain.from_iterable((v + '_x', v + '_y', v + '_z') for v in ['ee_pos']))
twist_headers = list(itertools.chain.from_iterable((v + '_x', v + '_y', v + '_z') for v in ['ee_twist_lin']))
command_twist_headers = list(itertools.chain.from_iterable((v + '_x', v + '_y', v + '_z') for v in ['des_twist_lin']))
desired_headers = ['timestamp'] + ft_headers + pos_headers + twist_headers + command_twist_headers

In [ ]:
def transform_velocity(df_row, headers_list):
    R = Rotation.from_quat(df_row[['ee_ori_x', 'ee_ori_y', 'ee_ori_z', 'ee_ori_w']]).as_matrix().transpose()
    for headers in headers_list:
        df_row[headers] = R.dot(df_row[headers].values)
    return df_row

def add_force_derivatives(df, headers):
    tmp_data = df.drop(data.index[-1])
    for header in headers:
        tmp_data[header + '_dot'] = [(b - a) / (d - c) for a, b, c, d in 
                                     zip(df[header][:-1], df[header][1:],
                                         df['timestamp'][:-1], df['timestamp'][1:])]
    return tmp_data

def plot_phases(data, headers, title=""):
    '''Plot data from x and y with subplots.'''
    fig = make_subplots(rows=len(headers), cols=1,x_title='time [s]',shared_xaxes=True)
    colors = [dict(color='blue'), dict(color='green'), dict(color='red'), dict(color='yellow')]
    
    for i, header in enumerate(headers):
        for phase in range(2):
            if i is 0:
                fig.append_trace(go.Scatter(
                    x=data['timestamp'][data['phase'] == phase],
                    y=data[header][data['phase'] == phase],
                    name='phase ' + str(phase),
                    line=colors[phase],
                ), row=i+1, col=1)
            else:
                fig.append_trace(go.Scatter(
                    x=data['timestamp'][data['phase'] == phase],
                    y=data[header][data['phase'] == phase],
                    showlegend=False,
                    line=colors[phase],
                ), row=i+1, col=1)
        if 'force' in header:
            fig.update_yaxes(title_text=header + ' [N]', row=i+1, col=1)
        elif 'twist' in header:
            fig.update_yaxes(title_text=header + ' [m/s]', row=i+1, col=1)
        elif 'pos' in header:
            fig.update_yaxes(title_text=header + ' [m]', row=i+1, col=1)

    fig.update_layout(height=300, width=600, title_text=title)
    fig.show()


In [ ]:
segmented_runs = {}
for cq, runs in all_runs.items():
    for r in runs:
        if r in ['20210218_orange_cut_04_0.050000_0.020000_0.050000_0.005000.csv',
                 '20210218_apple_cut_30_0.070000_0.005000_0.050000_0.005000.csv',
                 '20210218_orange_cut_32_0.070000_0.006000_0.040000_0.005000.csv',
                 '20210218_orange_cut_33_0.070000_0.005000_0.050000_0.005000.csv']:
            continue
            
        print('Processing run ' + r)
        all_data = pd.read_csv(join(data_folder, r))
        
        time_step = [(b - a) for a, b in zip(all_data['timestamp'][:-1], all_data['timestamp'][1:])]
        if len(np.where(np.asarray(time_step) > 0.5)[0]):
            time_step_jumps = [x[0] for x in np.where(np.asarray(time_step) > 0.5)]
            all_data = all_data.drop(all_data.index[:time_step_jumps[0] + 1])
            all_data = all_data.reset_index()
        del time_step
        
        # reset timestamp
        all_data['timestamp'] -= all_data['timestamp'].iloc[0]
        
        # transform velocity to ee frame
        all_data = all_data.apply(lambda x: transform_velocity(x, [twist_headers, command_twist_headers]), axis=1)
        
        # get cut indices
        #des_twist_lin_norm = data.apply(lambda x: x[command_twist_headers].mean(), axis=1)
        #start_cut_idx = data[des_twist_lin_norm == 0].index[-1]
        #end_cut_idx = data.iloc[start_cut_idx:][data['ee_force_x'].iloc[start_cut_idx:] > -0.2].index[0]
        
        # get cut center at cut start
        #cut_radius = float(r.split('_')[-1][:-4])
        #cut_start = all_data.iloc[start_cut_idx]
        #R = Rotation.from_quat(cut_start[['ee_ori_x', 'ee_ori_y', 'ee_ori_z', 'ee_ori_w']]).as_matrix()
        #cut_center_world_frame = R.dot([0, -cut_radius, 0]) + cut_start[pos_headers].values
        #print(cut_center_world_frame)  
        #del cut_start, R
        
        data = all_data[desired_headers]
        del all_data
        
        if filter_data:
            freq = len(data.index) / (data['timestamp'].iloc[-1] - data['timestamp'].iloc[0])
            data[ft_headers] = filter_data(data[ft_headers], freq, cutoff_freq=5)
            data[twist_headers] = filter_data(data[twist_headers], freq, cutoff_freq=15)
        del freq
        
        nb_desired_samples = desired_freq * (data['timestamp'].iloc[-1] - data['timestamp'].iloc[0])
        time_vector = pd.Series(np.linspace(data['timestamp'].iloc[0], data['timestamp'].iloc[-1], int(nb_desired_samples)))
        data = downsample(data, time_vector, data_average, data.columns)
        del nb_desired_samples, time_vector
        
        des_twist_lin_norm = data.apply(lambda x: x[command_twist_headers].mean(), axis=1)
        start_cut_idx = data[des_twist_lin_norm == 0].index[-1]
        end_cut_idx = data.iloc[start_cut_idx:][data['des_twist_lin_z'].iloc[start_cut_idx:] < -0.01].index[0]
        data['phase'] = [0] * start_cut_idx + [1] * (end_cut_idx - start_cut_idx) + [2] * (len(data.index) - end_cut_idx)
        del des_twist_lin_norm
        
        cut_radius = float(r.split('_')[-1][:-4])
        cut_center = [0, cut_radius, 0]
        cut_position = pd.DataFrame()
        for i, header in enumerate(pos_headers):
            cut_position[header] = data[header][data['phase'] == 1]
            cut_position[header] -= cut_position[header].iloc[0]
        cut_position[pos_headers] = cut_position[pos_headers].sub(cut_center)
        #plt.plot(data['timestamp'][data['phase'] == 1], cut_position['ee_pos_x'])
        #plt.plot(data['timestamp'][data['phase'] == 1], cut_position['ee_pos_y'])
        #plt.show()
        displacement = [cut_radius * np.arctan2(x, -y) for x, y in zip(cut_position['ee_pos_x'], cut_position['ee_pos_y'])]
        #plt.plot(data['timestamp'][data['phase'] == 1], displacement)
        #plt.show()
        
        #displacement = [np.linalg.norm(x) for x in
        #                    zip(cut_position['ee_pos_x'],
        #                        cut_position['ee_pos_y'],
        #                        cut_position['ee_pos_z'])]
        
        data['displacement'] = np.nan
        data['displacement'].iloc[start_cut_idx:end_cut_idx] = displacement
        del displacement, start_cut_idx, end_cut_idx
        
        data['label'] = [fruit_label] * len(data.index)
        
        segmented_runs[r] = data
        
        #plot_phases(data, command_twist_headers)
        export_run(r, data, join('segmented_data', experiment, fruit))

        
# this is just for development purposes because the above loops take too much time
pickle_out = open('segmented_cuts_' + fruit + '.pickle', 'wb')
pickle.dump(segmented_runs, pickle_out)
pickle_out.close()
        

In [ ]:
pickle_in = open('segmented_cuts_' + fruit + '.pickle', 'rb')
segmented_runs = pickle.load(pickle_in)

In [ ]:
headers = ft_headers
fig = make_subplots(rows=len(headers), cols=1,x_title='time [s]',shared_xaxes=True)
colors = {'0.015000': [dict(color='blue'), False], 
          '0.005000': [dict(color='green'), False],
          '0.004000': [dict(color='red'), False], 
          '0.003000': [dict(color='yellow'), False],
          '0.002500': [dict(color='black'), False]}

for k, name in enumerate(segmented_runs.keys()):
    #if name.split('_')[5] != '0.005000':
    #    continue
    #print(name)
    
    
    color = colors[name.split('_')[5]][0]
    legend_showed = colors[name.split('_')[5]][1]
    run = segmented_runs[name].copy()
    #plot_phases(run, headers)
    for i, header in enumerate(headers):
        if i is 0 and not legend_showed:
            colors[name.split('_')[5]][1] = True
            fig.append_trace(go.Scatter(
                x=run['displacement'][run['phase'] == 1],
                y=-run[header][run['phase'] == 1],
                name=name.split('_')[5],
                line=color,
            ), row=i+1, col=1)
        else:
            fig.append_trace(go.Scatter(
                x=run['displacement'][run['phase'] == 1],
                y=-run[header][run['phase'] == 1],
                showlegend=False,
                line=color,
            ), row=i+1, col=1)
        if 'force' in header:
            fig.update_yaxes(title_text=header + ' [N]', row=i+1, col=1)
        if 'twist' in header:
            fig.update_yaxes(title_text=header + ' [m/s]', row=i+1, col=1)
        

fig.update_layout(height=600, width=800, title_text=fruit)
fig.show()

In [ ]:
#headers = ['ee_force_z', 'ee_twist_lin_z', 'des_twist_lin_z']
headers = ft_headers
fig = make_subplots(rows=len(headers), cols=1,x_title='time [s]',shared_xaxes=True)
colors = [dict(color='blue'), dict(color='green'), dict(color='red'), dict(color='yellow')]

for k, name in enumerate(segmented_runs.keys()):
    if name == '20210218_orange_cut_32_0.070000_0.006000_0.040000_0.005000.csv':
        continue
    run = segmented_runs[name].copy()
    start = run[run['phase'] == 1].index[0]
    run['timestamp'] -= run['timestamp'][run['phase'] == 0].iloc[-1]
    #plot_phases(run, headers)
    for i, header in enumerate(headers):
        for phase in range(4):
            if i is 0 and k is 0:
                fig.append_trace(go.Scatter(
                    x=run['timestamp'][run['phase'] == phase],
                    y=run[header][run['phase'] == phase],
                    name='phase ' + str(phase),
                    line=colors[phase],
                ), row=i+1, col=1)
            else:
                fig.append_trace(go.Scatter(
                    x=run['timestamp'][run['phase'] == phase],
                    y=run[header][run['phase'] == phase],
                    showlegend=False,
                    line=colors[phase],
                ), row=i+1, col=1)
        if 'force' in header:
            fig.update_yaxes(title_text=header + ' [N]', row=i+1, col=1)
        if 'twist' in header:
            fig.update_yaxes(title_text=header + ' [m/s]', row=i+1, col=1)
        

fig.update_layout(height=600, width=800, title_text=fruit)
fig.show()
    

In [ ]:
headers = command_twist_headers
fig = make_subplots(rows=len(headers), cols=1,x_title='time [s]',shared_xaxes=True)
colors = [dict(color='blue'), dict(color='green'), dict(color='red'), dict(color='yellow')]

for k, name in enumerate(segmented_runs.keys()):
    run = segmented_runs[name].copy()
    run['timestamp'] -= run['timestamp'][run['phase'] == 0].iloc[-2]
    #plot_phases(run, headers)
    for i, header in enumerate(headers):
        for phase in range(4):
            if i is 0 and k is 0:
                fig.append_trace(go.Scatter(
                    x=run['timestamp'][run['phase'] == phase],
                    y=run[header][run['phase'] == phase],
                    name='phase ' + str(phase),
                    line=colors[phase],
                ), row=i+1, col=1)
            else:
                fig.append_trace(go.Scatter(
                    x=run['timestamp'][run['phase'] == phase],
                    y=run[header][run['phase'] == phase],
                    showlegend=False,
                    line=colors[phase],
                ), row=i+1, col=1)
        if 'force' in header:
            fig.update_yaxes(title_text=header + ' [N]', row=i+1, col=1)
        if 'twist' in header:
            fig.update_yaxes(title_text=header + ' [m/s]', row=i+1, col=1)
        

fig.update_layout(height=600, width=800, title_text=fruit)
fig.show()
    